In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt
from datetime import date, timedelta
from dateutil import relativedelta

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
# establish connection
engine = create_engine("sqlite:///Facilities.sqlite")
conn = engine.connect()

In [6]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# reflect the tables
Base.metadata.tables #['LosAnglesCountyLicData']

immutabledict({'LosAnglesCountyLicData': Table('LosAnglesCountyLicData', MetaData(bind=None), Column('OSHPD_ID', INTEGER(), table=<LosAnglesCountyLicData>, primary_key=True, nullable=False), Column('FACILITY_NAME', TEXT(), table=<LosAnglesCountyLicData>), Column('LICENSE_NUM', INTEGER(), table=<LosAnglesCountyLicData>), Column('FACILITY_LEVEL_DESC', TEXT(), table=<LosAnglesCountyLicData>), Column('DBA_ADDRESS1', TEXT(), table=<LosAnglesCountyLicData>), Column('DBA_CITY', TEXT(), table=<LosAnglesCountyLicData>), Column('DBA_ZIP_CODE', INTEGER(), table=<LosAnglesCountyLicData>), Column('COUNTY_CODE', INTEGER(), table=<LosAnglesCountyLicData>), Column('COUNTY_NAME', TEXT(), table=<LosAnglesCountyLicData>), Column('ER_SERVICE_LEVEL_DESC', TEXT(), table=<LosAnglesCountyLicData>), Column('TOTAL_NUMBER_BEDS', INTEGER(), table=<LosAnglesCountyLicData>), Column('FACILITY_STATUS_DESC', TEXT(), table=<LosAnglesCountyLicData>), Column('FACILITY_STATUS_DATE', TEXT(), table=<LosAnglesCountyLicData>)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['LosAnglesCountyLicData',
 'encounters',
 'hosp_la_df',
 'hospitals',
 'hospitals_avg_encounters']

In [8]:
# Save references to each table
Hospitals = Base.classes.hospitals
Licensed  = Base.classes.LosAnglesCountyLicData

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

## Facitlities Analysis

In [10]:
# this methhod much more fun but might not work with all SQL syntaxes (but I prefer this way!)
query_stations = 'SELECT * FROM LosAnglesCountyLicData WHERE LICENSE_CATEGORY_DESC = "Community Clinic"'

# so easy to put it into a pandas dataframe!
df = pd.read_sql_query(query_stations, con=engine)
df

,OSHPD_ID,FACILITY_NAME,LICENSE_NUM,FACILITY_LEVEL_DESC,DBA_ADDRESS1,DBA_CITY,DBA_ZIP_CODE,COUNTY_CODE,COUNTY_NAME,ER_SERVICE_LEVEL_DESC,TOTAL_NUMBER_BEDS,FACILITY_STATUS_DESC,FACILITY_STATUS_DATE,LICENSE_TYPE_DESC,LICENSE_CATEGORY_DESC,LATITUDE,LONGITUDE
0,306174012,"PLANNED PARENTHOOD BASICS, LAKEWOOD",550001424,Parent Facility,5519 Del Amo Blvd,Lakewood,90713,19,Los Angeles,Not Applicable,None,Open,11/18/2010,Clinic,Community Clinic,33.84714,-118.12431
1,306190007,TICHENOR ORTHOPEDIC CLINIC FOR CHILDREN,960000004,Parent Facility,1660 Termino Ave,Long Beach,90804,19,Los Angeles,Not Applicable,None,Open,3/24/1947,Clinic,Community Clinic,33.78781,-118.14588
2,306190127,EISNER PEDIATRIC AND FAMILY MEDICAL CENTER,960000030,Parent Facility,1530 S Olive St,Los Angeles,90015,19,Los Angeles,Not Applicable,None,Open,3/23/1961,Clinic,Community Clinic,34.03540,-118.26522
3,306190169,SOUTH BAY CHILDRENS HEALTH CENTER,960000038,Parent Facility,410 CAMINO REAL,REDONDO BEACH,90277,19,Los Angeles,Not Applicable,None,Open,3/15/1954,Clinic,Community Clinic,33.83744,-118.38175
4,306190306,THE SABAN COMMUNITY CLINIC - BEVERLY HEALTH CE...,960000578,Parent Facility,8405 Beverly Blvd,Los Angeles,90048,19,Los Angeles,Not Applicable,None,Open,10/8/1968,Clinic,Community Clinic,34.07608,-118.37347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,306198432,NORTHEAST VALLEY HEALTH CORPORATION - VAN NUYS...,960000947,Parent Facility,14624 Sherman Way,Van Nuys,91405,19,Los Angeles,Not Applicable,None,Open,5/9/2017,Clinic,Community Clinic,34.20115,-118.45170
373,306198476,SERENITY CARE HEALTH GROUP,550003904,Parent Facility,2100 E Anaheim St,Long Beach,90804,19,Los Angeles,Not Applicable,None,Open,6/28/2017,Clinic,Community Clinic,33.78256,-118.16636
374,306198492,NOBLEQUEST HEALTH FOUNDATION CLINIC,550003956,Parent Facility,14435 Hamlin St,Van Nuys,91401,19,Los Angeles,Not Applicable,None,Open,8/14/2017,Clinic,Community Clinic,34.18869,-118.44771
375,306198494,VIA CARE 607,550003985,Parent Facility,607 S Atlantic Blvd,Los Angeles,90022,19,Los Angeles,Not Applicable,None,Open,10/16/2017,Clinic,Community Clinic,34.02478,-118.15603


## Hospitals with ER Analysis

In [11]:
# this methhod much more fun but might not work with all SQL syntaxes (but I prefer this way!)
query_stations = 'SELECT * FROM LosAnglesCountyLicData WHERE LICENSE_TYPE_DESC = "Hospital" AND ER_SERVICE_LEVEL_DESC like "%Emergency%"'

# so easy to put it into a pandas dataframe!
df = pd.read_sql_query(query_stations, con=engine)
df

,OSHPD_ID,FACILITY_NAME,LICENSE_NUM,FACILITY_LEVEL_DESC,DBA_ADDRESS1,DBA_CITY,DBA_ZIP_CODE,COUNTY_CODE,COUNTY_NAME,ER_SERVICE_LEVEL_DESC,TOTAL_NUMBER_BEDS,FACILITY_STATUS_DESC,FACILITY_STATUS_DATE,LICENSE_TYPE_DESC,LICENSE_CATEGORY_DESC,LATITUDE,LONGITUDE
0,106190017,ALHAMBRA HOSPITAL MEDICAL CENTER,930000005,Parent Facility,100 S RAYMOND AVE,ALHAMBRA,91801,19,Los Angeles,Emergency - Basic,144,Open,1/1/1946,Hospital,General Acute Care Hospital,34.089880,-118.144900
1,106190034,ANTELOPE VALLEY HOSPITAL,930000008,Parent Facility,1600 W AVE J,LANCASTER,93534,19,Los Angeles,Emergency - Basic,420,Open,9/27/1955,Hospital,General Acute Care Hospital,34.687800,-118.157981
2,106190045,CATALINA ISLAND MEDICAL CENTER,930000010,Parent Facility,100 FALLS CANYON RD,AVALON,90704,19,Los Angeles,Emergency - Standby,12,Open,1/1/1946,Hospital,General Acute Care Hospital,33.338870,-118.333690
3,106190053,ST. MARY MEDICAL CENTER - LONG BEACH,930000012,Parent Facility,1050 Linden Ave,Long Beach,90813,19,Los Angeles,Emergency - Basic,360,Open,1/1/1946,Hospital,General Acute Care Hospital,33.780238,-118.186641
4,106190066,LOS ANGELES COMMUNITY HOSPITAL AT BELLFLOWER,930000039,Consolidated Facility,9542 Artesia Blvd,Bellflower,90706,19,Los Angeles,Emergency - Basic,144,Open,7/23/2015,Hospital,General Acute Care Hospital,33.874720,-118.129450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,106191231,LOS ANGELES COUNTY OLIVE VIEW-UCLA MEDICAL CENTER,60000133,Parent Facility,14445 OLIVE VIEW DR,SYLMAR,91342,19,Los Angeles,Emergency - Basic,355,Open,10/20/1959,Hospital,General Acute Care Hospital,34.324180,-118.452550
70,106191450,KAISER FOUNDATION HOSPITAL - WOODLAND HILLS,930000358,Parent Facility,5601 DE SOTO AVE,WOODLAND HILLS,91367,19,Los Angeles,Emergency - Basic,274,Open,4/8/1986,Hospital,General Acute Care Hospital,34.171980,-118.588290
71,106196035,KAISER FOUNDATION HOSPITAL - BALDWIN PARK,930000920,Parent Facility,1011 BALDWIN PARK BLVD,BALDWIN PARK,91706,19,Los Angeles,Emergency - Basic,272,Open,10/7/1998,Hospital,General Acute Care Hospital,34.063480,-117.986000
72,106196403,KAISER FOUNDATION HOSPITAL - DOWNEY,930000078,Parent Facility,9333 Imperial Hwy,Downey,90242,19,Los Angeles,Emergency - Basic,352,Open,8/31/2009,Hospital,General Acute Care Hospital,33.917123,-118.129022


## Encounters Analysis

In [12]:
# this methhod much more fun but might not work with all SQL syntaxes (but I prefer this way!)
query_stations = 'SELECT * FROM encounters'

# so easy to put it into a pandas dataframe!
df = pd.read_sql_query(query_stations, con=engine)
df

,year,oshpd_id,facility_name,county_name,er_service_level_desc,type,count
0,2012,106190017,ALHAMBRA HOSPITAL MEDICAL CENTER,LOS ANGELES,BASIC,ED_Admit,2603.0
1,2012,106190017,ALHAMBRA HOSPITAL MEDICAL CENTER,LOS ANGELES,BASIC,ED_Visit,10263.0
2,2012,106190034,ANTELOPE VALLEY HOSPITAL,LOS ANGELES,BASIC,ED_Admit,13234.0
3,2012,106190034,ANTELOPE VALLEY HOSPITAL,LOS ANGELES,BASIC,ED_Visit,91791.0
4,2012,106190045,CATALINA ISLAND MEDICAL CENTER,LOS ANGELES,STANDBY,ED_Admit,15.0
...,...,...,...,...,...,...,...
1083,2018,106196168,MEMORIALCARE MILLER CHILDREN'S & WOMEN'S HOSPI...,LOS ANGELES,None,ED_Visit,NaN
1084,2018,106196403,KAISER FOUNDATION HOSPITAL - DOWNEY,LOS ANGELES,BASIC,ED_Admit,8038.0
1085,2018,106196403,KAISER FOUNDATION HOSPITAL - DOWNEY,LOS ANGELES,BASIC,ED_Visit,97457.0
1086,2018,106196405,PALMDALE REGIONAL MEDICAL CENTER,LOS ANGELES,BASIC,ED_Admit,6607.0


## Avg Encounters Analysis

In [13]:
# this methhod much more fun but might not work with all SQL syntaxes (but I prefer this way!)
query_stations = 'SELECT * FROM hospitals_avg_encounters'

# so easy to put it into a pandas dataframe!
df = pd.read_sql_query(query_stations, con=engine)
df

,OSHPD_ID,FAC_NAME,DBA_ADDRESS1,DBA_CITY,DBA_ZIP_CODE,ER_SERVICE_LEVEL_DESC,TOTAL_NUMBER_BEDS,LATITUDE,LONGITUDE,TYPE_CNTRL,NET_TOT,er_service_level,AvgAdmits,AvgVisits
0,106190017,ALHAMBRA HOSPITAL MEDICAL CENTER,100 S RAYMOND AVE,ALHAMBRA,91801,Emergency - Basic,144,34.089880,-118.144900,Investor - Ptnr.,192170249,BASIC,3131.57,13445.71
1,106190034,ANTELOPE VALLEY HOSPITAL,1600 W AVE J,LANCASTER,93534,Emergency - Basic,420,34.687800,-118.157981,District,442828351,BASIC,12443.29,105222.29
2,106190045,CATALINA ISLAND MEDICAL CENTER,100 FALLS CANYON RD,AVALON,90704,Emergency - Standby,12,33.338870,-118.333690,Non Profit Corp.,9724231,STANDBY,8.43,1916.86
3,106190053,ST. MARY MEDICAL CENTER - LONG BEACH,1050 Linden Ave,Long Beach,90813,Emergency - Basic,360,33.780238,-118.186641,Non Profit Corp.,352700261,BASIC,6538.29,48016.43
4,106190081,BEVERLY HOSPITAL,309 W BEVERLY BLVD,MONTEBELLO,90640,Emergency - Basic,202,34.016140,-118.101320,Non Profit Corp.,179845974,BASIC,6743.14,28261.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,106194219,KECK HOSPITAL OF USC,1500 SAN PABLO ST,LOS ANGELES,90033,None,401,34.062840,-118.202870,Investor - Corp.,1178217757,None,1.00,NaN
73,106196035,KAISER FOUNDATION HOSPITAL - BALDWIN PARK,1011 BALDWIN PARK BLVD,BALDWIN PARK,91706,Emergency - Basic,272,34.063480,-117.986000,Non Profit Corp.,137703960,BASIC,6265.57,73186.14
74,106196168,MEMORIALCARE MILLER CHILDREN'S & WOMEN'S HOSPI...,2801 Atlantic Ave,Long Beach,90806,None,357,33.808070,-118.185060,Non Profit Corp.,495886562,None,1.00,NaN
75,106196403,KAISER FOUNDATION HOSPITAL - DOWNEY,9333 Imperial Hwy,Downey,90242,Emergency - Basic,352,33.917123,-118.129022,Non Profit Corp.,188443936,BASIC,9002.86,86981.86
